<a href="https://colab.research.google.com/github/UznetDev/Data-science-home-work/blob/main/24_Avg_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQL**

### Import librares

In [1]:
import pandas as pd

### Set-up for SQL

In [2]:
%load_ext sql
%sql sqlite://

##### Load dataset

In [3]:
monarchs = pd.read_csv('monarchs.csv')
presidents = pd.read_csv('presidents.csv')
prime_minister_terms = pd.read_csv('prime_minister_terms.csv')
prime_ministers = pd.read_csv('prime_ministers.csv')
states = pd.read_csv('states.csv')

##### Upload dataset to SQL

In [4]:
%sql DROP TABLE IF EXISTS monarchs;
%sql --persist monarchs

%sql DROP TABLE IF EXISTS presidents;
%sql --persist presidents

%sql DROP TABLE IF EXISTS prime_minister_terms;
%sql --persist prime_minister_terms

%sql DROP TABLE IF EXISTS prime_ministers;
%sql --persist prime_ministers

%sql DROP TABLE IF EXISTS states;
%sql --persist states

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


'Persisted states'

### *Class work*

#### Part 1

In [5]:
%%sql SELECT monarch, country
FROM monarchs
EXCEPT
SELECT prime_minister, country
FROM prime_ministers;

 * sqlite://
Done.


monarch,country
Harald V,Norway
Queen Elizabeth II,United Kingdom


In [6]:
%%sql SELECT country, continent, president
FROM presidents;

 * sqlite://
Done.


country,continent,president
Egypt,Africa,Abdel Fattah el-Sisi
Portugal,Europe,Marcelo Rebelo de Sousa
USA,North America,Joe Biden
Uruguay,South America,Luis Lacalle Pou
Pakistan,Asia,Arif Alvi
Chile,South America,Gabriel Boric
India,Asia,Ram Nath Kovind


In [7]:
%%sql SELECT country
FROM states
WHERE indep_year < 1800;

 * sqlite://
Done.


country
Portugal
USA


In [8]:
%%sql SELECT president, country, continent
FROM presidents
WHERE country IN (
    SELECT country
    FROM states
    WHERE indep_year < 1800
);

 * sqlite://
Done.


president,country,continent
Marcelo Rebelo de Sousa,Portugal,Europe
Joe Biden,USA,North America


In [9]:
%%sql SELECT president, s.country, s.continent
FROM presidents as p
INNER JOIN states as s
ON p.country = s.country
WHERE s.indep_year < 1800;

 * sqlite://
Done.


president,country,continent
Marcelo Rebelo de Sousa,Portugal,Europe
Joe Biden,USA,North America


In [10]:
%%sql SELECT president, s.country, s.continent
FROM presidents as p
INNER JOIN states as s
ON p.country != s.country
WHERE s.indep_year < 1800;

 * sqlite://
Done.


president,country,continent
Abdel Fattah el-Sisi,Portugal,Europe
Joe Biden,Portugal,Europe
Luis Lacalle Pou,Portugal,Europe
Arif Alvi,Portugal,Europe
Gabriel Boric,Portugal,Europe
Ram Nath Kovind,Portugal,Europe
Abdel Fattah el-Sisi,USA,North America
Marcelo Rebelo de Sousa,USA,North America
Luis Lacalle Pou,USA,North America
Arif Alvi,USA,North America


In [11]:
%%sql SELECT country, president
FROM presidents
WHERE continent LIKE '%America'
AND country NOT IN (
    SELECT country
    FROM states
    WHERE indep_year < 1800
);

 * sqlite://
Done.


country,president
Uruguay,Luis Lacalle Pou
Chile,Gabriel Boric


In [12]:
%%sql SELECT p.country, p.president
FROM presidents p
INNER JOIN states s ON p.country = s.country
WHERE p.continent LIKE '%America'
AND (s.indep_year >= 1800 OR s.indep_year IS NULL);

 * sqlite://
Done.


country,president
Uruguay,Luis Lacalle Pou
Chile,Gabriel Boric


In [13]:
%%sql SELECT DISTINCT continent
FROM states;

 * sqlite://
Done.


continent
Oceania
Asia
South America
Africa
Europe
North America


In [14]:
%%sql SELECT DISTINCT continent,
      (SELECT COUNT(*)
      FROM monarchs
      WHERE states.continent = monarchs.continent) AS num_monarchs
FROM states;

 * sqlite://
Done.


continent,num_monarchs
Oceania,0
Asia,2
South America,0
Africa,0
Europe,2
North America,0


In [15]:
%%sql SELECT s.continent, COUNT(*) AS num_monarchs
FROM monarchs AS m
LEFT JOIN states AS s
USING(country)
GROUP BY m.continent;

 * sqlite://
Done.


continent,num_monarchs
Asia,2
Europe,2


#### Part 2

In [16]:
%%sql SELECT
    continent,
    MAX(indep_year) AS most_recent
FROM states
GROUP BY continent;

 * sqlite://
Done.


continent,most_recent
Africa,1922
Asia,1984
Europe,1905
North America,1776
Oceania,1901
South America,1825


In [17]:
%%sql SELECT DISTINCT m.continent,
      sub.most_recent
FROM monarchs AS m,
      (SELECT continent,
      MAX(indep_year) AS most_recent
      FROM states
      GROUP BY continent) AS sub
WHERE m.continent = sub.continent
ORDER BY m.continent;

 * sqlite://
Done.


continent,most_recent
Asia,1984
Europe,1905


In [18]:
%%sql SELECT continent,
      MAX(indep_year) AS most_recent
FROM states
GROUP BY continent;

 * sqlite://
Done.


continent,most_recent
Africa,1922
Asia,1984
Europe,1905
North America,1776
Oceania,1901
South America,1825


In [19]:
%%sql
SELECT T2.continent, T1.country
FROM states AS T1 INNER JOIN
    (SELECT continent, MAX(indep_year) AS max_indep
    FROM states
    WHERE continent IN (SELECT DISTINCT continent FROM monarchs)
    GROUP BY continent) AS T2 ;

 * sqlite://
Done.


continent,country
Asia,Australia
Europe,Australia
Asia,Brunei
Europe,Brunei
Asia,Chile
Europe,Chile
Asia,Egypt
Europe,Egypt
Asia,India
Europe,India
